---
<a id='1'></a>
## 1️⃣ 환경 설정

워크샵에서 사용한 설정을 로드합니다.

In [16]:
# 환경 설정 로드
import subprocess
import json
import os
from dotenv import load_dotenv

load_dotenv()

# 설정 로드 (워크샵 노트북에서 사용한 값)
RESOURCE_GROUP = os.getenv("RESOURCE_GROUP", "rg-mcp-lab")
APIM_NAME = os.getenv("APIM_SERVICE_NAME", "")
FUNCTION_APP_NAME = os.getenv("FUNCTION_APP_NAME", "")

# Azure CLI에서 정보 가져오기
try:
    account_result = subprocess.run(
        ['az', 'account', 'show', '-o', 'json'],
        capture_output=True, text=True, check=True
    )
    account = json.loads(account_result.stdout)
    SUBSCRIPTION_ID = account['id']
    TENANT_ID = account['tenantId']
    print("✅ Azure 계정 연결됨")
    print(f"   구독: {account['name']}")
    print(f"   구독 ID: {SUBSCRIPTION_ID}")
    print(f"   테넌트 ID: {TENANT_ID}")
except:
    print("❌ Azure CLI에 로그인되어 있지 않습니다.")
    print("   az login 명령으로 로그인하세요.")

print()
print("📌 삭제 대상 리소스:")
print(f"   리소스 그룹: {RESOURCE_GROUP}")

# 리소스 그룹의 리소스 목록 조회
try:
    resources_result = subprocess.run([
        'az', 'resource', 'list',
        '--resource-group', RESOURCE_GROUP,
        '--query', '[].{name:name, type:type}',
        '-o', 'json'
    ], capture_output=True, text=True)
    
    if resources_result.returncode == 0:
        resources = json.loads(resources_result.stdout)
        print()
        print(f"   포함된 리소스 ({len(resources)}개):")
        for r in resources:
            resource_type = r['type'].split('/')[-1]
            print(f"     - {r['name']} ({resource_type})")
    else:
        print(f"   ⚠️ 리소스 조회 실패: {resources_result.stderr}")
except Exception as e:
    print(f"   ⚠️ 리소스 조회 오류: {e}")

✅ Azure 계정 연결됨
   구독: ME-MngEnvMCAP843676-hyungilkim-1
   구독 ID: a9737323-45d3-45da-94ff-cffe6e5c6f79
   테넌트 ID: 38bd4dc5-c85c-4efd-a73a-3dfe7d6f632e

📌 삭제 대상 리소스:
   리소스 그룹: rg-mcp-lab

   포함된 리소스 (7개):
     - stmcpdevelopment56e69a (storageAccounts)
     - mcp-dev-law-i6ht (workspaces)
     - mcp-dev-asp-i6ht (serverFarms)
     - mcp-dev-ai-i6ht (components)
     - mcp-dev-func-i6ht (sites)
     - Application Insights Smart Detection (actiongroups)
     - mcp-dev-apim-i6ht (service)


---
<a id='2'></a>
## 2️⃣ 리소스 그룹 삭제

⚠️ **주의**: 이 명령은 리소스 그룹과 그 안의 모든 리소스를 **영구적으로** 삭제합니다.

In [17]:
# 리소스 그룹 삭제
import subprocess

print(f"⚠️  리소스 그룹 삭제: {RESOURCE_GROUP}")
print()
print("삭제될 리소스:")
print("  - API Management 서비스")
print("  - Azure Functions App")
print("  - Application Insights")
print("  - Storage Account")
print("  - Log Analytics Workspace")
print("  - 모든 연결된 리소스")
print()

# 확인 메시지
confirmation = input(f"정말로 '{RESOURCE_GROUP}' 리소스 그룹을 삭제하시겠습니까? (yes/no): ")

if confirmation.lower() == 'yes':
    print()
    print("🗑️  리소스 그룹 삭제 중... (이 작업은 몇 분 소요될 수 있습니다)")
    
    try:
        result = subprocess.run(
            [
                "az", "group", "delete",
                "--name", RESOURCE_GROUP,
                "--yes",
                "--no-wait"
            ],
            capture_output=True,
            text=True,
            check=True
        )
        
        print("✅ 리소스 그룹 삭제 요청이 제출되었습니다.")
        print()
        print("삭제 상태 확인:")
        print(f"   az group show --name {RESOURCE_GROUP}")
        print()
        print("또는 Azure Portal에서 확인:")
        print(f"   https://portal.azure.com/#view/HubsExtension/BrowseResourceGroups")
        
    except subprocess.CalledProcessError as e:
        print(f"❌ 삭제 실패: {e.stderr}")
        print()
        print("수동으로 삭제하려면:")
        print(f"   az group delete --name {RESOURCE_GROUP} --yes")
else:
    print()
    print("❌ 삭제가 취소되었습니다.")
    print()
    print("💡 Tip: 리소스를 유지하려면 이 셀을 실행하지 마세요.")

⚠️  리소스 그룹 삭제: rg-mcp-lab

삭제될 리소스:
  - API Management 서비스
  - Azure Functions App
  - Application Insights
  - Storage Account
  - Log Analytics Workspace
  - 모든 연결된 리소스


🗑️  리소스 그룹 삭제 중... (이 작업은 몇 분 소요될 수 있습니다)
✅ 리소스 그룹 삭제 요청이 제출되었습니다.

삭제 상태 확인:
   az group show --name rg-mcp-lab

또는 Azure Portal에서 확인:
   https://portal.azure.com/#view/HubsExtension/BrowseResourceGroups


---
<a id='2.5'></a>
## 2.5 Soft-Deleted APIM 완전 삭제 (Purge)

리소스 그룹 삭제 후 APIM은 **48시간 동안 soft-deleted 상태**로 유지됩니다.  
동일한 이름으로 새 APIM을 생성하려면 먼저 Purge해야 합니다.

**REST API 사용**: [Deleted Services - Purge API](https://learn.microsoft.com/en-us/rest/api/apimanagement/deleted-services/purge?view=rest-apimanagement-2024-05-01)

In [19]:
# Soft-deleted APIM 확인 및 Purge (REST API 사용)
import subprocess
import json
import hashlib
import requests
import time

# APIM 이름 및 위치 설정
location = os.getenv("LOCATION", "koreacentral")

# 워크샵에서 사용한 APIM 이름 (mcp-dev-apim-i6ht)
apim_name = os.getenv("APIM_SERVICE_NAME", "")
if not apim_name:
    # 기본값: 리소스 그룹 해시 기반 이름
    unique_suffix = hashlib.md5(RESOURCE_GROUP.encode()).hexdigest()[:4]
    apim_name = f"mcp-dev-apim-{unique_suffix}"

print("=" * 60)
print("🗑️  Soft-Deleted APIM 리소스 확인 및 Purge")
print("=" * 60)
print()

# Azure 액세스 토큰 획득
token_result = subprocess.run([
    'az', 'account', 'get-access-token',
    '--resource', 'https://management.azure.com/',
    '--query', 'accessToken',
    '--output', 'tsv'
], capture_output=True, text=True)

if token_result.returncode != 0:
    print(f"❌ 액세스 토큰 획득 실패: {token_result.stderr}")
    raise Exception("Azure 인증이 필요합니다. 'az login'을 실행하세요.")

access_token = token_result.stdout.strip()

print(f"Subscription ID: {SUBSCRIPTION_ID}")
print()

# API Management REST API 설정
api_version = "2024-05-01"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Purge 함수 정의
def purge_apim(svc_name, svc_location):
    """APIM을 Purge합니다."""
    # 위치 정규화 (예: "Korea Central" -> "koreacentral")
    normalized_location = svc_location.lower().replace(' ', '')
    purge_url = f"https://management.azure.com/subscriptions/{SUBSCRIPTION_ID}/providers/Microsoft.ApiManagement/locations/{normalized_location}/deletedservices/{svc_name}?api-version={api_version}"
    
    print(f"   🔄 Purge 요청 중: {svc_name} ({svc_location})...")
    purge_response = requests.delete(purge_url, headers=headers)
    
    if purge_response.status_code in [200, 202, 204]:
        print(f"   ✅ Purge 성공! (HTTP {purge_response.status_code})")
        return True
    else:
        print(f"   ❌ Purge 실패! (HTTP {purge_response.status_code})")
        print(f"      {purge_response.text[:200]}")
        return False

# 1. Soft-deleted APIM 목록 조회
list_url = f"https://management.azure.com/subscriptions/{SUBSCRIPTION_ID}/providers/Microsoft.ApiManagement/deletedservices?api-version={api_version}"

print("🔍 Soft-deleted APIM 목록 조회 중...")
list_response = requests.get(list_url, headers=headers)

if list_response.status_code == 200:
    deleted_services = list_response.json().get('value', [])
    
    if deleted_services:
        print(f"\n📋 발견된 Soft-deleted APIM ({len(deleted_services)}개):")
        print("-" * 60)
        for i, svc in enumerate(deleted_services, 1):
            name = svc.get('name', 'N/A')
            loc = svc.get('location', 'N/A')
            del_date = svc.get('properties', {}).get('deletionDate', 'N/A')
            purge_date = svc.get('properties', {}).get('scheduledPurgeDate', 'N/A')
            print(f"  {i}. {name}")
            print(f"     위치: {loc}")
            print(f"     삭제일: {del_date[:10] if del_date != 'N/A' else 'N/A'}")
            print(f"     자동 Purge 예정: {purge_date[:10] if purge_date != 'N/A' else 'N/A'}")
            print()
        print("-" * 60)
        
        # 삭제 옵션 선택
        print()
        print("삭제 옵션:")
        print("  all  - 모든 soft-deleted APIM Purge")
        print("  1,2  - 번호로 선택 (쉼표로 구분)")
        print("  no   - 취소")
        print()
        
        choice = input("선택: ").strip().lower()
        
        if choice == 'all':
            print()
            print("🗑️  모든 APIM Purge 시작...")
            success_count = 0
            for svc in deleted_services:
                if purge_apim(svc.get('name'), svc.get('location')):
                    success_count += 1
            print()
            print(f"✅ Purge 완료: {success_count}/{len(deleted_services)}개 성공")
            
        elif choice != 'no' and choice:
            try:
                # 번호로 선택
                indices = [int(x.strip()) - 1 for x in choice.split(',')]
                print()
                print("🗑️  선택한 APIM Purge 시작...")
                success_count = 0
                for idx in indices:
                    if 0 <= idx < len(deleted_services):
                        svc = deleted_services[idx]
                        if purge_apim(svc.get('name'), svc.get('location')):
                            success_count += 1
                    else:
                        print(f"   ⚠️ 잘못된 번호: {idx + 1}")
                print()
                print(f"✅ Purge 완료: {success_count}개 성공")
            except ValueError:
                print("❌ 잘못된 입력입니다.")
        else:
            print("❌ Purge가 취소되었습니다.")
        
        # Purge 후 대기
        if choice not in ['no', '']:
            print()
            print("⏳ 리소스 정리 대기 중... (30초)")
            time.sleep(30)
            print("✅ 대기 완료")
    else:
        print("✅ Soft-deleted APIM이 없습니다.")
        print("   삭제 대기 중인 APIM이 없습니다.")
else:
    print(f"⚠️ Soft-deleted APIM 조회 실패!")
    print(f"   HTTP Status: {list_response.status_code}")
    print(f"   Response: {list_response.text}")

print()
print("=" * 60)

🗑️  Soft-Deleted APIM 리소스 확인 및 Purge

Subscription ID: a9737323-45d3-45da-94ff-cffe6e5c6f79

🔍 Soft-deleted APIM 목록 조회 중...

📋 발견된 Soft-deleted APIM (1개):
------------------------------------------------------------
  1. mcp-dev-apim-i6ht
     위치: Korea Central
     삭제일: 2025-12-11
     자동 Purge 예정: 2025-12-13

------------------------------------------------------------

삭제 옵션:
  all  - 모든 soft-deleted APIM Purge
  1,2  - 번호로 선택 (쉼표로 구분)
  no   - 취소


🗑️  선택한 APIM Purge 시작...
   🔄 Purge 요청 중: mcp-dev-apim-i6ht (Korea Central)...
   ✅ Purge 성공! (HTTP 202)

✅ Purge 완료: 1개 성공

⏳ 리소스 정리 대기 중... (30초)
✅ 대기 완료



---
<a id='3'></a>
## 3️⃣ 개별 리소스 삭제 (선택사항)

리소스 그룹을 유지하면서 특정 리소스만 삭제하고 싶은 경우 사용합니다.

In [ ]:
# 개별 리소스 삭제 명령어 생성
print("🗑️  개별 리소스 삭제 명령어:")
print()

print("=" * 80)
print("1️⃣ API Management 서비스 삭제 (가장 비용이 많이 드는 리소스)")
print("=" * 80)
if APIM_NAME:
    print(f"az apim delete \\")
    print(f"  --name {APIM_NAME} \\")
    print(f"  --resource-group {RESOURCE_GROUP} \\")
    print(f"  --yes --no-wait")
else:
    print("# APIM_SERVICE_NAME이 설정되지 않았습니다.")
    print("# az apim list --resource-group {RESOURCE_GROUP} 로 확인하세요.")
print()
print("💰 비용 절감: ~$50-100/month (Basicv2 SKU 기준)")
print()

print("=" * 80)
print("2️⃣ Functions App 삭제")
print("=" * 80)
if FUNCTION_APP_NAME:
    print(f"az functionapp delete \\")
    print(f"  --name {FUNCTION_APP_NAME} \\")
    print(f"  --resource-group {RESOURCE_GROUP}")
else:
    print("# FUNCTION_APP_NAME이 설정되지 않았습니다.")
    print("# az functionapp list --resource-group {RESOURCE_GROUP} 로 확인하세요.")
print()

print("=" * 80)
print("3️⃣ Storage Account 삭제")
print("=" * 80)
print(f"# Storage Account 이름 확인:")
print(f"az storage account list --resource-group {RESOURCE_GROUP} --query '[].name' -o tsv")
print()
print(f"# 삭제 명령:")
print(f"az storage account delete \\")
print(f"  --name <STORAGE_ACCOUNT_NAME> \\")
print(f"  --resource-group {RESOURCE_GROUP} \\")
print(f"  --yes")
print()

print("=" * 80)
print("4️⃣ Application Insights 삭제")
print("=" * 80)
print(f"# Application Insights 이름 확인:")
print(f"az monitor app-insights component list --resource-group {RESOURCE_GROUP} --query '[].name' -o tsv")
print()
print(f"# 삭제 명령:")
print(f"az monitor app-insights component delete \\")
print(f"  --app <APP_INSIGHTS_NAME> \\")
print(f"  --resource-group {RESOURCE_GROUP}")
print()

print("=" * 80)
print("5️⃣ Log Analytics Workspace 삭제")
print("=" * 80)
print(f"# Log Analytics Workspace 이름 확인:")
print(f"az monitor log-analytics workspace list --resource-group {RESOURCE_GROUP} --query '[].name' -o tsv")
print()
print(f"# 삭제 명령:")
print(f"az monitor log-analytics workspace delete \\")
print(f"  --workspace-name <WORKSPACE_NAME> \\")
print(f"  --resource-group {RESOURCE_GROUP} \\")
print(f"  --yes")
print()

print("✅ 개별 리소스 삭제 명령어 출력 완료")
print()
print("💡 Tip: 위의 명령어를 복사하여 터미널에서 실행하세요.")
print("⚠️  주의: APIM 삭제는 복구 불가능하며, 재생성 시 시간이 오래 걸립니다 (30-40분).")

---
<a id='4'></a>
## 4️⃣ Entra ID 앱 삭제

워크샵에서 생성한 Entra ID (Azure AD) 앱 등록을 삭제합니다.

In [20]:
# Entra ID 앱 삭제
import subprocess
import json

print("🔐 Entra ID 앱 등록 조회 중...")
print()

# MCP 관련 앱 목록 조회
try:
    apps_result = subprocess.run([
        'az', 'ad', 'app', 'list',
        '--filter', "startswith(displayName, 'MCP-')",
        '--query', '[].{appId:appId, displayName:displayName}',
        '-o', 'json'
    ], capture_output=True, text=True)
    
    if apps_result.returncode == 0:
        apps = json.loads(apps_result.stdout)
        
        if apps:
            print(f"🔍 발견된 MCP 관련 앱 ({len(apps)}개):")
            for app in apps:
                print(f"   - {app['displayName']} ({app['appId']})")
            
            print()
            confirmation = input("이 앱들을 삭제하시겠습니까? (yes/no): ")
            
            if confirmation.lower() == 'yes':
                for app in apps:
                    print(f"\n🗑️  삭제 중: {app['displayName']}...")
                    delete_result = subprocess.run([
                        'az', 'ad', 'app', 'delete',
                        '--id', app['appId']
                    ], capture_output=True, text=True)
                    
                    if delete_result.returncode == 0:
                        print(f"   ✅ 삭제 완료")
                    else:
                        print(f"   ❌ 삭제 실패: {delete_result.stderr[:100]}")
                
                print("\n✅ Entra ID 앱 삭제 완료!")
            else:
                print("\n❌ 삭제가 취소되었습니다.")
        else:
            print("ℹ️  삭제할 MCP 관련 앱이 없습니다.")
    else:
        print(f"❌ 앱 조회 실패: {apps_result.stderr}")
        
except Exception as e:
    print(f"❌ 오류: {e}")

print()
print("💡 수동으로 삭제하려면:")
print("   1. Azure Portal > Entra ID > App registrations")
print("   2. MCP-API, MCP-Client 앱 선택")
print("   3. Delete 클릭")

🔐 Entra ID 앱 등록 조회 중...

🔍 발견된 MCP 관련 앱 (2개):
   - MCP-API- (56a56150-850b-4889-8bab-cc775012d4ba)
   - MCP-Client- (3c1271cc-b9a6-44b3-9ccc-0145f9bdba1e)


🗑️  삭제 중: MCP-API-...
   ✅ 삭제 완료

🗑️  삭제 중: MCP-Client-...
   ✅ 삭제 완료

✅ Entra ID 앱 삭제 완료!

💡 수동으로 삭제하려면:
   1. Azure Portal > Entra ID > App registrations
   2. MCP-API, MCP-Client 앱 선택
   3. Delete 클릭


---
<a id='5'></a>
## 5️⃣ 비용 관리 팁

Azure 리소스를 유지하면서 비용을 최소화하는 방법입니다.

In [ ]:
# 비용 관리 팁
print("💰 Azure 비용 절감 가이드:")
print()

print("=" * 80)
print("1️⃣ API Management 비용 최적화")
print("=" * 80)
print("""
현재 SKU: Basicv2 (~$50-100/month)

비용 절감 옵션:
  ⭐ Developer SKU로 변경: ~$50/month
     - 개발/테스트 환경에 적합
     - SLA 없음, 스케일링 제한
     - 명령어: az apim update --name <NAME> --sku-name Developer

  ⭐ Consumption 플랜 고려:
     - 사용량 기반 과금 (첫 100만 건 무료)
     - Cold start 발생 가능
     - 낮은 트래픽 환경에 적합

  ⚠️  불필요한 APIM 기능 비활성화:
     - 사용하지 않는 API 제거
     - 불필요한 정책 최소화
""")
print()

print("=" * 80)
print("2️⃣ Azure Functions 비용 최적화")
print("=" * 80)
print("""
현재 플랜: Consumption (사용량 기반 과금)

비용 절감 팁:
  ✅ 실행 시간 최소화:
     - 불필요한 코드 제거
     - 효율적인 알고리즘 사용
     - 외부 API 호출 캐싱

  ✅ 메모리 할당 최적화:
     - 필요한 만큼만 메모리 사용
     - 불필요한 라이브러리 제거

  ✅ 무료 할당량 활용:
     - 월 100만 실행 무료
     - 400,000 GB-s 무료
""")
print()

print("=" * 80)
print("3️⃣ Storage & Monitoring 비용 최적화")
print("=" * 80)
print("""
비용 절감 팁:
  ✅ Log 보관 정책 설정:
     - Application Insights: 90일 → 30일
     - Storage Account: 오래된 로그 자동 삭제
     - Log Analytics: 샘플링 비율 조정

  ✅ 불필요한 진단 로그 비활성화:
     - 개발 완료 후 상세 로깅 수준 낮추기
     - Verbose 로그 → Warning 이상만

  ✅ Storage Account 타입 최적화:
     - Hot → Cool 티어 (로그용)
     - LRS 복제 전략 (개발 환경)
""")
print()

print("=" * 80)
print("4️⃣ 비용 모니터링 설정")
print("=" * 80)
print("""
✅ Cost Management 활용:
   Azure Portal > Cost Management + Billing > Cost analysis

✅ 예산 경고 설정:
   az consumption budget create \\
     --budget-name "monthly-budget" \\
     --amount 100 \\
     --time-grain Monthly \\
     --resource-group <RESOURCE_GROUP>

✅ 비용 추적:
   - 일일 비용 확인
   - 예상치 못한 증가 알림 설정
   - 리소스별 비용 분석
""")
print()

print("=" * 80)
print("5️⃣ 개발 환경 자동화")
print("=" * 80)
print("""
✅ 스케줄 기반 시작/중지:
   - 업무 시간에만 APIM 실행
   - Azure Automation 활용
   - 야간/주말 자동 중지

✅ 환경 분리:
   - 개발 환경: Developer SKU + Consumption
   - 운영 환경: Standard SKU + Premium Functions
""")
print()

print("✅ 비용 관리 가이드 완료!")
print()
print("📊 현재 예상 비용:")
print("   - APIM Basicv2: ~$60/month")
print("   - Functions Consumption: ~$5/month (낮은 사용량 가정)")
print("   - Storage + Monitoring: ~$10/month")
print("   - 총 예상: ~$75/month")
print()
print("💡 Tip: Azure Cost Calculator로 정확한 비용 예측 가능")
print("   https://azure.microsoft.com/pricing/calculator/")

---

## ✅ 정리 완료

리소스 정리가 완료되었습니다.

### 🔍 삭제 상태 확인

```bash
# 리소스 그룹 상태 확인
az group show --name rg-mcp-lab

# Entra ID 앱 확인
az ad app list --filter "startswith(displayName, 'MCP-')" --query '[].displayName'
```

### 📝 참고사항

- **APIM Soft Delete**: APIM은 삭제 후 48시간 동안 soft delete 상태로 유지됩니다.
  - 완전 삭제: `az apim deletedservice purge --service-name <NAME> --location <LOCATION>`
  
- **Entra ID 앱 복구**: 삭제된 앱은 30일 이내 복구 가능합니다.
  - Azure Portal > Entra ID > App registrations > Deleted applications

---

**감사합니다!** 🙏